In [1]:
import tifffile
import skimage
import scipy
import napari

from napari_animation import Animation
from napari_animation.easing import Easing
from moviepy.editor import VideoFileClip
from PIL import Image, ImageSequence

import numpy as np
from scipy import ndimage as ndi
from skimage import data

In [2]:
def convert_mp4_to_gif(mp4_file, gif_file):
    down_scale = 1
    clip = VideoFileClip(mp4_file)
    
    # 计算新尺寸
    width = clip.size[0] // down_scale
    height = clip.size[1] // down_scale
    
    clip_resized = clip.resize((width, height))
    clip_resized.write_gif(gif_file)
    
def get_and_save_gif(name):
    viewer = napari.Viewer(ndisplay=3)
    rotated_img = tifffile.imread(name+'.tif').transpose(1,2,0)
    name = name + '_rotated'
    tifffile.imsave(name +'.tif', rotated_img)
    
    viewer.open(name+'.tif')  
    animation = Animation(viewer)

    viewer.camera.zoom = 2
    animation.capture_keyframe()

    viewer.camera.angles = (0.0, 90.0, 90.0)
    animation.capture_keyframe(steps=400)

    animation.animate(name+".mp4", canvas_only=True, fps=60)

    # 指定输入的MP4文件和输出的GIF文件名
    mp4_file = name+".mp4"
    gif_file = name+".gif"

    convert_mp4_to_gif(mp4_file, gif_file)

In [11]:
# name = 'rotated_dirty'
# get_and_save_gif('dataset/outputtest')

# get_and_save_gif('dataset/inputtest')

Rendering frames...


100%|██████████| 401/401 [00:13<00:00, 29.04it/s]


MoviePy - Building file dataset/inputtest_rotated.gif with imageio.


生成并合并gif

In [5]:
def save_video_and_gif(name0):
    
    rotated_img = tifffile.imread(name0+'.tif').transpose(1,2,0)
    name0 = name0 + '_rotated'
    tifffile.imsave(name0 +'.tif', rotated_img)
    
    viewer = napari.Viewer(ndisplay=3)
    viewer.open(name0+'.tif')  
    animation = Animation(viewer)

    viewer.camera.zoom = 2
    animation.capture_keyframe()

    viewer.camera.angles = (0.0, 90.0, 90.0)
    animation.capture_keyframe(steps=100)

    animation.animate(name0+".mp4", canvas_only=True, fps=30)

    # 指定输入的MP4文件和输出的GIF文件名
    mp4_file = name0+".mp4"
    gif_file = name0+".gif"

    convert_mp4_to_gif(mp4_file, gif_file)


def crop_gif(input_path, output_path):
    # 打开 GIF 文件
    gif = Image.open(input_path)

    # 获取 GIF 的总帧数和每帧的尺寸
    frames = []
    for frame in ImageSequence.Iterator(gif):
        frames.append(frame.copy())

    frame_width, frame_height = frames[0].size

    # 计算左右 1/5 到 4/5 的区域
    start_x = int(frame_width / 6)
    end_x = int(frame_width * 5 / 6)

    # 创建一个存储剪裁帧的列表
    cropped_frames = []

    # 循环遍历每一帧，将剪裁的部分添加到列表中
    for frame in frames:
        cropped_frame = frame.crop((start_x, 0, end_x, frame_height))
        cropped_frames.append(cropped_frame)

    # 将剪裁后的帧保存为新的 GIF 文件
    cropped_frames[0].save(output_path, save_all=True, append_images=cropped_frames[1:], optimize=False, loop=0)
    
def merge_gifs_horizontally(gif1_path, gif2_path, output_path):
    # 打开两个GIF图像文件
    gif1 = Image.open(gif1_path)
    gif2 = Image.open(gif2_path)

    # 获取GIF图像的帧数
    gif1_frames = gif1.n_frames
    gif2_frames = gif2.n_frames

    # 确保两个GIF图像的帧数相同，取较小的帧数作为最终合并后的帧数
    num_frames = min(gif1_frames, gif2_frames)

    # 创建一个列表来存储合并后的每一帧图像
    frames = []

    for i in range(num_frames):
        # 选择两个GIF图像对应的帧
        gif1.seek(i)
        gif2.seek(i)

        # 将两个帧按水平方向合并
        merged_frame = Image.new('RGB', (gif1.width + gif2.width, max(gif1.height, gif2.height)))
        merged_frame.paste(gif1, (0, 0))
        merged_frame.paste(gif2, (gif1.width, 0))

        # 添加合并后的帧到帧列表中
        frames.append(merged_frame)

    # 保存合并后的GIF图像
    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0)


In [7]:
from PIL import Image, ImageSequence


save_video_and_gif('inputtest')
save_video_and_gif('outputtest')

# 调用函数剪裁 GIF 并保存
crop_gif('inputtest_rotated.gif', 'inputtest_rotated.gif')
crop_gif('outputtest_rotated.gif', 'outputtest_rotated.gif')

merge_gifs_horizontally("inputtest_rotated.gif", "outputtest_rotated.gif", "res.gif")


Rendering frames...


100%|██████████| 101/101 [00:03<00:00, 30.10it/s]


MoviePy - Building file inputtest_rotated.gif with imageio.


Rendering frames...


100%|██████████| 101/101 [00:03<00:00, 30.97it/s]


MoviePy - Building file outputtest_rotated.gif with imageio.
